# def get_dict_attr(obj, attr)
## 方法解析顺序（MRO）
```python
class A: ...
class B(A): ...
class C(A): ...
class D(B, C): ...

print(D.mro())
# 输出: [<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]
```
## 源码
对于 `obj`（实例/类型），**直接**在其自身和继承链的所有 `__dict__` 中查找 `attr` 属性。
```python
def get_dict_attr(obj, attr):
    if inspect.isclass(obj):
        cls = obj  
    else:
        cls = obj.__class__  
    
    # obj 可能是实例也可能是类型
    for obj in [obj] + cls.mro():
        if attr in obj.__dict__:
            return obj.__dict__[attr]  
    
    raise AttributeError
```

# def default_getattr_func
获取 `obj.attr`，根据其是否可被调用返回 `obj.attr(*args, **kwargs)/obj.attr`。

```python
def default_getattr_func(obj: tp.Any,
                         attr: str,
                         args: tp.Optional[tp.Args] = None,
                         kwargs: tp.Optional[tp.Kwargs] = None,
                         call_attr: bool = True) -> tp.Any:
    if args is None:
        args = () 
    if kwargs is None:
        kwargs = {}
    
    out = getattr(obj, attr)
    
    if callable(out) and call_attr:
        return out(*args, **kwargs)
    
    return out
```

# def deep_getattr
对 `obj` 执行*属性方法链*`attr_chain`：
- 基本情况有三种：
  - `无.string`
  - (`无.string`, `tuple`)
  - (`无.string`, `tuple`, `dict`)

  都是查找 `obj` 的对应属性，根据其是否可调用返回属性调用结果/属性

- 每一项递归。例如：
  - `[('method', (args,)), 'attr']——>obj.method(*args).attr`
```python
def deep_getattr(obj: tp.Any,
                 attr_chain: tp.Union[str, tuple, Iterable],
                 getattr_func: tp.Callable = default_getattr_func,
                 call_last_attr: bool = True) -> tp.Any:
    checks.assert_instance_of(attr_chain, (str, tuple, Iterable))

    if isinstance(attr_chain, str):
        if '.' in attr_chain:
            return deep_getattr(
                obj,
                attr_chain.split('.'), 
                getattr_func=getattr_func,
                call_last_attr=call_last_attr
            )
        return getattr_func(obj, attr_chain, call_attr=call_last_attr)
    
    if isinstance(attr_chain, tuple):
        if len(attr_chain) == 1 \
                and isinstance(attr_chain[0], str):
            return getattr_func(obj, attr_chain[0])
        
        if len(attr_chain) == 2 \
                and isinstance(attr_chain[0], str) \
                and isinstance(attr_chain[1], tuple):
            return getattr_func(obj, attr_chain[0], args=attr_chain[1])
        
        if len(attr_chain) == 3 \
                and isinstance(attr_chain[0], str) \
                and isinstance(attr_chain[1], tuple) \
                and isinstance(attr_chain[2], dict):
            return getattr_func(obj, attr_chain[0], args=attr_chain[1], kwargs=attr_chain[2])
    
    result = obj  
    for i, attr in enumerate(attr_chain):
        if i < len(attr_chain) - 1:
            result = deep_getattr(
                result,
                attr,
                getattr_func=getattr_func,
                call_last_attr=True
            )
        else:
            result = deep_getattr(
                result,
                attr,
                getattr_func=getattr_func,
                call_last_attr=call_last_attr
            )
    return result
```

# class AttrResolver

## 使用方法
被某类 `A` 所继承，`A` 需重写 `self_aliases, resolve_self, pre_resolve_attr, post_resolve_attr` 方法。

核心是方法 `resolve_attr`：
```python
a.resolve_attr(attr, 
                args, 
                cond_kwargs, 
                kwargs, 
                custom_arg_names, 
                cache_dct, 
                use_caching, 
                passed_kwargs_out)
```
（1）将 **attr** 和 **cond_kwargs** $ \cup $ **kwargs** 作为参数调用 `pre_resolve_attr`，获得返回的 `_attr`
  
（2）如果 dir(type(self)) 中包含 `get_{attr}`，则将 `_attr` 置为它。如果 getattr(type(self), _attr) 是 Method/Function
- attr_func = self.\_attr（相当于进行了绑定）
-  attr_func_kwargs=(**cond_kwargs** $ \cup $ **kwargs**) $ \cap $ (get_{**attr**}的非可变非关键字参数 $ \cup $ **kwargs**)，并且存到**passed_kwargs_out**
- 是否 **use_caching** 并且 attr_func_kwargs $ \cap $ **custom_arg_names** $  =  \emptyset $ 并且 attr in **cache_dct**
- 是：out = cache_dct[attr]
- 否：out = `attr_func(*args, **attr_func_kwargs)`
    - 如果 **use_caching** 并且 attr_func_kwargs $ \cap $ **custom_arg_names** $  =  \emptyset $：cache_dct[attr] = out

（3）非（2）的情况，是否 **use_caching** 并且 attr in **cache_dct**
  - 是：out = cache_dct[attr]
  - 否：out = `self._attr`
    - 如果 **use_caching**：cache_dct[attr] = out

（4）将 **attr**，out，**cond_kwargs** $ \cup $ **kwargs** 作为参数调用 `post_resolve_attr`，然后返回

## 源码

### self_aliases
定义当前对象的所有别名。须子类重写。
```python
@property
def self_aliases(self) -> tp.Set[str]:
    return {'self'}
```

### resolve_self
解析当前对象实例，可能根据参数创建新实例。须子类重写。
```python
def resolve_self(self: AttrResolverT,
                    cond_kwargs: tp.KwargsLike = None,
                    custom_arg_names: tp.Optional[tp.Set[str]] = None,
                    impacts_caching: bool = True,
                    silence_warnings: bool = False) -> AttrResolverT:
    return self
```

### pre_resolve_attr 和 post_resolve_attr
供函数 `resolve_attr` 使用。须子类重写。
```python
def pre_resolve_attr(self, attr: str, final_kwargs: tp.KwargsLike = None) -> str:
    return attr

def post_resolve_attr(self, attr: str, out: tp.Any, final_kwargs: tp.KwargsLike = None) -> str:
    return out
```

### resolve_attr
```python
def resolve_attr(self,
                    attr: str,
                    args: tp.ArgsLike = None,
                    cond_kwargs: tp.KwargsLike = None,
                    kwargs: tp.KwargsLike = None,
                    custom_arg_names: tp.Optional[tp.Container[str]] = None,
                    cache_dct: tp.KwargsLike = None,
                    use_caching: bool = True,
                    passed_kwargs_out: tp.KwargsLike = None) -> tp.Any:

    if custom_arg_names is None:
        custom_arg_names = list()
    if cache_dct is None:
        cache_dct = {}
    if args is None:
        args = ()
    if kwargs is None:
        kwargs = {}
    if passed_kwargs_out is None:
        passed_kwargs_out = {}
    final_kwargs = merge_dicts(cond_kwargs, kwargs)

    # Resolve attribute
    cls = type(self)
    _attr = self.pre_resolve_attr(attr, final_kwargs=final_kwargs)
    if 'get_' + attr in dir(cls):
        _attr = 'get_' + attr
    if inspect.ismethod(getattr(cls, _attr)) or inspect.isfunction(getattr(cls, _attr)):
        attr_func = getattr(self, _attr)
        attr_func_kwargs = dict()
        attr_func_arg_names = get_func_arg_names(attr_func)
        custom_k = False
        for k, v in final_kwargs.items():
            if k in attr_func_arg_names or k in kwargs:
                if k in custom_arg_names:
                    custom_k = True
                attr_func_kwargs[k] = v
                passed_kwargs_out[k] = v
        if use_caching and not custom_k and attr in cache_dct:
            out = cache_dct[attr]
        else:
            out = attr_func(*args, **attr_func_kwargs)
            if use_caching and not custom_k:
                cache_dct[attr] = out
    else:
        if use_caching and attr in cache_dct:
            out = cache_dct[attr]
        else:
            out = getattr(self, _attr)
            if use_caching:
                cache_dct[attr] = out
    out = self.post_resolve_attr(attr, out, final_kwargs=final_kwargs)
    return out
```